In [2]:
import importlib.util
import matplotlib.dates as md
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

# if importlib.util.find_spec('seaborn') is not None:
#     import seaborn as sns
#     sns.set()

## Load data

##### Toll price timeseries

In [3]:
# - Read in toll price timeseries data
df = pd.read_csv('/opt/dssg-hot/data/toll_prices/concatenated_405_prices_2018.csv')

# - Examine data
df.head(2)

,Time,2689,2690,2691,2692,2693,2694,2695,2696,2697,...,2769,2770,2771,2772,2773,2774,2731,2732,2733,2746
0,2018-01-01 05:01:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01 05:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Add and groupby useful computed time columns

In [4]:
df['new_dates'] = pd.to_datetime(df['Time'])
df['weekday_num'] = df['new_dates'].apply(lambda t: t.weekday())
df['month_num'] = df['new_dates'].apply(lambda t: t.month)
df['hour_num'] = df['new_dates'].apply(lambda t: t.hour)
df['time_of_day'] = df['new_dates'].apply(lambda t: t.time())
# NOTE: mon = 0, fri = 4; jan = 1, dec = 12

In [5]:
monthxtimeofday_grouped = df.groupby(['month_num','time_of_day'])
monthxtimeofday_stds = monthxtimeofday_grouped.aggregate(np.std)
monthxtimeofday_meds = monthxtimeofday_grouped.aggregate(np.median)

## Define fxns to plot toll price timeseries in diff ways

#### Plot toll prices over the day by month

In [19]:
def plot_tp_over_day_by_month(nb_trip_def_id1,nb_trip_def_id2,sb_trip_def_id1,sb_trip_def_id2):
    fig, axs = plt.subplots(4, 3, figsize=(16,12), sharex=True, sharey=True)
    
    isp_x = 0; isp_y = -1;
    for month in range(1,13):
        if (month!=5) and (month!=9):
            isp_y = isp_y + 1;
        elif month==5:
            isp_y = 0;
            isp_x = 1;
        elif month==9:
            isp_y = 0;
            isp_x = 2;

        axs[isp_y][isp_x].plot(
            df[df['month_num']==month]['time_of_day'],
            df[df['month_num']==month][nb_trip_def_id1],
            'C1.')
        axs[isp_y][isp_x].plot(
            df[df['month_num']==month]['time_of_day'],
            df[df['month_num']==month][nb_trip_def_id2],
            'C1.')
        axs[isp_y][isp_x].plot(
            df[df['month_num']==month]['time_of_day'],
            df[df['month_num']==month][sb_trip_def_id1],
            'C2.')
        axs[isp_y][isp_x].plot(
            df[df['month_num']==month]['time_of_day'],
            df[df['month_num']==month][sb_trip_def_id2],
            'C2.')

        axs[isp_y][isp_x].set_title('Month number: %i' %month)
        axs[isp_y][isp_x].yaxis.grid(True)
        axs[isp_y][isp_x].xaxis.grid(True)
        axs[isp_y][isp_x].tick_params(axis='x', labelrotation=90)

#### Plot toll prices over the day by day of the week

In [11]:
def plot_tp_over_day_by_day_of_week(nb_trip_def_id1,nb_trip_def_id2,sb_trip_def_id1,sb_trip_def_id2):
    fig, axs = plt.subplots(2, 3, figsize=(16,6), sharex=True, sharey=True)
    isp_x = 0; isp_y = -1;
    for day in range(0,6): # plot friday twice
        if (day!=2) and (day!=4):
            isp_y = isp_y + 1;
        elif day==2:
            isp_y = 0;
            isp_x = 1;
        elif day==4:
            isp_y = 0;
            isp_x = 2;

        if day==5: # plot friday twice
            day = 4;

        axs[isp_y][isp_x].plot(
            df[df['weekday_num']==day]['time_of_day'],
            df[df['weekday_num']==day][nb_trip_def_id1],
            'C1.')
        axs[isp_y][isp_x].plot(
            df[df['weekday_num']==day]['time_of_day'],
            df[df['weekday_num']==day][nb_trip_def_id2],
            'C1.')
        axs[isp_y][isp_x].plot(
            df[df['weekday_num']==day]['time_of_day'],
            df[df['weekday_num']==day][sb_trip_def_id1],
            'C2.')
        axs[isp_y][isp_x].plot(
            df[df['weekday_num']==day]['time_of_day'],
            df[df['weekday_num']==day][sb_trip_def_id2],
            'C2.')

        axs[isp_y][isp_x].set_title('Week day number: %i' %day)
        axs[isp_y][isp_x].yaxis.grid(True)
        axs[isp_y][isp_x].xaxis.grid(True)
        axs[isp_y][isp_x].tick_params(axis='x', labelrotation=90)

#### Plot toll prices over the day by day of the week w/ medians and stdevs

In [28]:
def plot_tp_over_day_by_month_medstd(nb_trip_def_id1,nb_trip_def_id2,sb_trip_def_id1,sb_trip_def_id2):
    fig, axs = plt.subplots(4, 3, figsize=(16,12), sharex=True, sharey=True)
    
    isp_x = 0; isp_y = -1;
    for month in range(1,13):
        if (month!=5) and (month!=9):
            isp_y = isp_y + 1;
        elif month==5:
            isp_y = 0;
            isp_x = 1;
        elif month==9:
            isp_y = 0;
            isp_x = 2;

        axs[isp_y][isp_x].plot(
                (monthxtimeofday_meds[nb_trip_def_id1][month]
             +monthxtimeofday_stds[nb_trip_def_id1][month]),'C1--')
        axs[isp_y][isp_x].plot(
            monthxtimeofday_meds[nb_trip_def_id1][month],'C1-')
        axs[isp_y][isp_x].plot(
            (monthxtimeofday_meds[nb_trip_def_id1][month]
             -monthxtimeofday_stds[nb_trip_def_id1][month]),'C1--')
        axs[isp_y][isp_x].plot(
            (monthxtimeofday_meds[nb_trip_def_id2][month]
             +monthxtimeofday_stds[nb_trip_def_id2][month]),'C1--')
        axs[isp_y][isp_x].plot(
            monthxtimeofday_meds[nb_trip_def_id2][month],'C1-')
        axs[isp_y][isp_x].plot(
            (monthxtimeofday_meds[nb_trip_def_id2][month]
             -monthxtimeofday_stds[nb_trip_def_id2][month]),'C1--')
        
        axs[isp_y][isp_x].plot(
            (monthxtimeofday_meds[sb_trip_def_id1][month]
             +monthxtimeofday_stds[sb_trip_def_id1][month]),'C2--')
        axs[isp_y][isp_x].plot(
            monthxtimeofday_meds[sb_trip_def_id1][month],'C2-')
        axs[isp_y][isp_x].plot(
            (monthxtimeofday_meds[sb_trip_def_id1][month]
             -monthxtimeofday_stds[sb_trip_def_id1][month]),'C2--')
        axs[isp_y][isp_x].plot(
            (monthxtimeofday_meds[sb_trip_def_id2][month]
             +monthxtimeofday_stds[sb_trip_def_id2][month]),'C2--')
        axs[isp_y][isp_x].plot(
            monthxtimeofday_meds[sb_trip_def_id2][month],'C2-')
        axs[isp_y][isp_x].plot(
            (monthxtimeofday_meds[sb_trip_def_id2][month]
             -monthxtimeofday_stds[sb_trip_def_id2][month]),'C2--')

        axs[isp_y][isp_x].set_title('Month number: %i' %month)
        axs[isp_y][isp_x].yaxis.grid(True)
        axs[isp_y][isp_x].xaxis.grid(True)
        axs[isp_y][isp_x].tick_params(axis='x', labelrotation=90)